In [6]:
from rdkit.Chem.rdmolfiles import MolFromPDBFile
from rdkit.Chem.rdchem import Mol
import numpy as np
import rdkit.Chem as Chem
from rdkit.Chem import AddHs, AssignStereochemistry, HybridizationType, ChiralType, BondStereo, MolFromMol2File
from rdkit.Chem.AllChem import ComputeGasteigerCharges
import os
import sys
sys.path.append("../../")
from src.data.pocket_utils import get_atom_coordinates, find_pocket_atoms_RDKit

In [7]:
def covalent_bonds(mol, atom_1, atom_2):
    """
    
    Determine if there is a covalent bond between two specified atoms in a molecule and get bond features.
    
    Parameters:
    mol (Chem.Mol): The RDKit molecule.
    atom_1 (int): The index of the first atom.
    atom_2 (int): The index of the second atom.
    
    Returns:
    tuple: A tuple containing bond order, aromaticity, conjugation, ring, and stereochemistry.
           If there is no bond, returns (0, 0, 0, 0, 0).
    
    """
    bond = mol.GetBondBetweenAtoms(atom_1, atom_2)

    if bond: 
        # Calculate edge features
        bond_order = bond.GetBondTypeAsDouble()
        aromaticity = int(bond.GetIsAromatic())
        conjugation = int(bond.GetIsConjugated())
        ring = int(bond.IsInRing())
        stereochemistry_tag = bond.GetStereo()
        
        if stereochemistry_tag == BondStereo.STEREOZ:
            stereochemistry = 1
        elif stereochemistry_tag == BondStereo.STEREOE:
            stereochemistry = -1
        else:
            stereochemistry = 0
    else:
        bond_order, aromaticity, conjugation, ring, stereochemistry = 0, 0, 0, 0, 0

    return (bond_order, aromaticity, conjugation, ring, stereochemistry)

In [8]:
mol = Chem.MolFromSmiles('CCO')
print(covalent_bonds(mol, 0, 1))  # Example output: (1.0, 0, 0, 0, 0)
print(covalent_bonds(mol, 0, 2))  # Example output: (0, 0, 0, 0, 0)

(1.0, 0, 0, 0, 0)
(0, 0, 0, 0, 0)


In [11]:
# Define test cases
test_cases = [
    ("CCO", 0, 1),  # Simple molecule, single bond
    ("CCO", 0, 2),  # Simple molecule, no direct bond
    ("c1ccccc1", 0, 1),  # Benzene, aromatic bond
    ("C=C", 0, 1),  # Ethene, double bond
    ("C#C", 0, 1),  # Acetylene, triple bond
    ("C1CCCCC1", 0, 1),  # Cyclohexane, single bond in ring
    ("C1CC1", 0, 2),  # Cyclopropane, single bond across ring
]


# Initialize RDKit molecules
molecules = [(Chem.MolFromSmiles(smiles), a1, a2) for smiles, a1, a2 in test_cases]



In [12]:

# Test the function with the test cases
for mol, atom_1, atom_2 in molecules:
    result = covalent_bonds(mol, atom_1, atom_2)
    print(f"SMILES: {Chem.MolToSmiles(mol)}, Atoms: ({atom_1}, {atom_2}) -> Bond Features: {result}")

SMILES: CCO, Atoms: (0, 1) -> Bond Features: (1.0, 0, 0, 0, 0)
SMILES: CCO, Atoms: (0, 2) -> Bond Features: (0, 0, 0, 0, 0)
SMILES: c1ccccc1, Atoms: (0, 1) -> Bond Features: (1.5, 1, 1, 1, 0)
SMILES: C=C, Atoms: (0, 1) -> Bond Features: (2.0, 0, 0, 0, 0)
SMILES: C#C, Atoms: (0, 1) -> Bond Features: (3.0, 0, 0, 0, 0)
SMILES: C1CCCCC1, Atoms: (0, 1) -> Bond Features: (1.0, 0, 0, 1, 0)
SMILES: C1CC1, Atoms: (0, 2) -> Bond Features: (1.0, 0, 0, 1, 0)


- **SMILES:** CCO, **Atoms:** (0, 1) -> **Bond Features:** (1.0, 0, 0, 0, 0)
- **SMILES:** CCO, **Atoms:** (0, 2) -> **Bond Features:** (0, 0, 0, 0, 0)
- **SMILES:** c1ccccc1, **Atoms:** (0, 1) -> **Bond Features:** (1.5, 1, 1, 1, 0)
- **SMILES:** C=C, **Atoms:** (0, 1) -> **Bond Features:** (2.0, 0, 1, 0, 0)
- **SMILES:** C#C, **Atoms:** (0, 1) -> **Bond Features:** (3.0, 0, 0, 0, 0)
- **SMILES:** C1CCCCC1, **Atoms:** (0, 1) -> **Bond Features:** (1.0, 0, 0, 1, 0)
- **SMILES:** C1CC1, **Atoms:** (0, 2) -> **Bond Features:** (1.0, 0, 0, 1, 0)
